# Pairing-Friendly Curves

**Module 07** | 07-pairings

*BN curves, embedding degree*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **pairing-friendly curves**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [The Weil Pairing](07b-weil-pairing-intuition.ipynb).
- Concepts and notation introduced in the previous notebook.

## Embedding Degree

In [ ]:
# The embedding degree k is the smallest k with r | p^k - 1
# TODO: compute embedding degree for sample curves

## BN Curves

In [ ]:
# Barreto-Naehrig curves: pairing-friendly with embedding degree 12
# TODO: parameterize a BN curve

## Security Considerations

In [ ]:
# Embedding degree affects both efficiency and security
# TODO: discuss the relationship between k and security level

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **pairing-friendly curves**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [BLS Signatures](07d-bls-signatures.ipynb)